## Goal
We want to explore how better is the performance of the system of generating a draft with a multiple step process rather than a single step process

### Initial Variables

In [10]:
import sys 
import os
import json_tricks as json
import time
sys.path.append("../../")
from backend.LLM.AnythingLLM_client import AnythingLLMClient
from backend.Generater import Generater
from backend.LLM.OllamaLLM import OllamaAI
from backend.database.schemas import Email


anyllm_client = AnythingLLMClient("http://localhost:3001/api", "3WMNAPZ-GYH4RBE-M67SR00-7Y7KYEF")
ollama_client = OllamaAI('http://localhost:11434', 'llama3:instruct')
generator = Generater(ollama_client, anyllm_client)
path_to_enquiry = "../../dataset/fake_email_1.json"
with open(path_to_enquiry, "r") as f:
    enquiry = json.load(f)

enquiry_text = enquiry["email"]
subject = enquiry_text.split("\n\n")[0]
body = '\n'.join(enquiry_text.split("\n\n")[1:])

Performing health check on http://localhost:11434
Ollama is running
Health check passed


### Single Step Process

In [7]:

def single_shot_prompt(email: Email):
    prompt = f""" You are David Frischer, the program administrator for the UCL Software Engineering program.
            You have received the following enquiry from a student:
            Subject: {email.subject}
            Body: {email.body}
            
           
            Draft a response to the student's email.
            """
    workspace_slug =  anyllm_client.get_workspace_slug("General")
    thread = anyllm_client.new_thread(workspace_slug, "tmp-thread")
    thread_slug = thread["slug"]
    start_time = time.time()
    response = anyllm_client.chat_with_thread(workspace_slug, thread_slug, prompt)
    total_time = time.time() - start_time
    anyllm_client.delete_thread(workspace_slug, thread_slug)
    return response, total_time

### Response format

## Mutiple Step Process

In [13]:
def multi_step_process(email:Email):
    start_time = time.time()
    response = generator.single_run_reply_to_email(email)
    response_time = time.time() - start_time
    
    return response, response_time
    

## Generate for multiple drafts

In [14]:
path_to_enquiries = "../../dataset"

counter = 0
results_folder = "./results"
os.makedirs(results_folder, exist_ok=True)
filepath = os.path.join(results_folder, "results.json")

results = {}
        
for enquiry_path in os.listdir(path_to_enquiries)[:2]:
    with open(os.path.join(path_to_enquiries, enquiry_path), "r") as f:
        enquiry = json.load(f)
        
    email = enquiry["email"]
    if "subject:" in email.lower():
        subject = email.split("\n")[0].replace("Subject:", "").strip()
        body = "\n".join(email.split("\n")[1:])
    else:
        subject = "Subject not found"
        body = email
    
    email = Email(subject=subject, body=body, id=counter)
    counter += 1
    
    response_single, response_time_single = single_shot_prompt(email)
    response_multi, response_time_multi = multi_step_process(email)
    
    results[str(email.id)] = {"single_shot": {"response": response_single, "response_time": response_time_single},
                              "multi_step": {"response": response_multi, "response_time": response_time_multi}}
    

with open(filepath, "w") as f:
    json.dump(results, f)
    
    
  


Creating new thread with name: tmp-thread in workspace: general


[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: test123 in workspace: general
Total Sim Distance: 0.4044093191623688
Total Sim Distance: 0.4109489321708679


[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: tmp-thread in workspace: general


[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)


Creating new thread with name: test123 in workspace: general
Total Sim Distance: 0.5420188158750534
Total Sim Distance: 0.6712368726730347
Total Sim Distance: 0.6257844865322113


[ERROR] Expecting value: line 1 column 1 (char 0)
[ERROR] Expecting value: line 1 column 1 (char 0)
